In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
! pip install pycaret

In [ ]:
! pip install h2o

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from pycaret.classification import *

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [414]:
train_orig = pd.read_csv('/content/drive/My Drive/Data/Train.csv')
test_orig = pd.read_csv('/content/drive/My Drive/Data/Test.csv')
ss = pd.read_excel('/content/drive/My Drive/Data/Sample_submission.xlsx')

In [415]:
train_orig.head()

,Team1,Team2,Stadium,HostCountry,Team1_Venue,Team2_Venue,Team1_Innings,Team2_Innings,MonthOfMatch,MatchWinner
0,5,4,37,4,Home,Away,Second,First,Dec,4
1,1,14,84,7,Neutral,Neutral,First,Second,Sep,1
2,9,15,47,9,Home,Away,First,Second,Feb,9
3,7,2,102,6,Home,Away,First,Second,Aug,2
4,6,8,46,5,Home,Away,First,Second,Aug,6


# Data Preprocessing

In [416]:
train = train_orig.copy()
test = test_orig.copy()

# Regarding Venue(Try Away with zero and different replacements)
train['Team1_Venue'] = train_orig['Team1_Venue'].replace({'Home':1, 'Away':-1, 'Neutral':0})
train['Team2_Venue'] = train_orig['Team2_Venue'].replace({'Home':1, 'Away':-1, 'Neutral':0})

test['Team1_Venue'] = test_orig['Team1_Venue'].replace({'Home':1, 'Away':-1, 'Neutral':0})
test['Team2_Venue'] = test_orig['Team2_Venue'].replace({'Home':1, 'Away':-1, 'Neutral':0})

# Team1_Innings can be changed to Batting first
train['Team1_Innings'] = train_orig['Team1_Innings'].replace({'Second':0,'First':1})
train['Team2_Innings'] = train_orig['Team2_Innings'].replace({'Second':0,'First':1})

test['Team1_Innings'] = test_orig['Team1_Innings'].replace({'Second':0,'First':1})
test['Team2_Innings'] = test_orig['Team2_Innings'].replace({'Second':0,'First':1})

month_map = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
train['MonthOfMatch'] = train_orig['MonthOfMatch'].replace(month_map)
test['MonthOfMatch'] = test_orig['MonthOfMatch'].replace(month_map)

# quarter_map = {'Jan':1,'Feb':1,'Mar':1,'Apr':2,'May':2,'Jun':2,'Jul':3,'Aug':3,'Sep':3,'Oct':4,'Nov':4,'Dec':4}

# train['QuarterOfMatch'] = train_orig['MonthOfMatch'].replace(quarter_map)
# test['QuarterOfMatch'] = test_orig['MonthOfMatch'].replace(quarter_map)

# semester_map = {'Jan':1,'Feb':1,'Mar':1,'Apr':1,'May':1,'Jun':1,'Jul':2,'Aug':2,'Sep':2,'Oct':2,'Nov':2,'Dec':2}
# train['SemOfMatch'] = train_orig['MonthOfMatch'].replace(semester_map)
# test['SemOfMatch'] = test_orig['MonthOfMatch'].replace(semester_map)


# Team1 is replaced by its winning percentage
train['Team1'] = train_orig['Team1'].replace(dict(train_orig['MatchWinner'].value_counts(normalize=True)*100))
train['Team2'] = train_orig['Team2'].replace(dict(train_orig['MatchWinner'].value_counts(normalize=True)*100))

test['Team1'] = test_orig['Team1'].replace(dict(train_orig['MatchWinner'].value_counts(normalize=True)*100))
test['Team2'] = test_orig['Team2'].replace(dict(train_orig['MatchWinner'].value_counts(normalize=True)*100))

train['DiffTeam'] = train['Team1'] - train['Team2']
test['DiffTeam'] = test['Team1'] - test['Team2']


In [419]:
label = []
for i in range(len(train)):
  record = train_orig.iloc[i]
  if record['Team1'] == record['MatchWinner']:
    label += [1]
  else:
    label += [0]

In [420]:
train['label'] = label
train.drop('MatchWinner',1,inplace=True)
train.head()

,Team1,Team2,Stadium,HostCountry,Team1_Venue,Team2_Venue,Team1_Innings,Team2_Innings,MonthOfMatch,DiffTeam,label
0,12.679426,9.449761,37,4,1,-1,0,1,12,3.229665,0
1,15.111643,10.446571,84,7,0,0,1,0,9,4.665072,1
2,8.891547,3.468900,47,9,1,-1,1,0,2,5.422648,1
3,0.956938,2.791069,102,6,1,-1,1,0,8,-1.834131,0
4,1.315789,0.598086,46,5,1,-1,1,0,8,0.717703,1


Label 0 that means winning Team 2 was not able to capture properly. That can be noted from the confusion matrix

In [421]:
label0_df = train[train['label'] == 0]
label0_df.head()

,Team1,Team2,Stadium,HostCountry,Team1_Venue,Team2_Venue,Team1_Innings,Team2_Innings,MonthOfMatch,DiffTeam,label
0,12.679426,9.449761,37,4,1,-1,0,1,12,3.229665,0
3,0.956938,2.791069,102,6,1,-1,1,0,8,-1.834131,0
9,9.449761,12.679426,21,3,1,-1,0,1,8,-3.229665,0
12,8.891547,12.599681,103,9,1,-1,1,0,2,-3.708134,0
13,12.679426,12.599681,75,4,1,-1,1,0,4,0.079745,0


In [445]:
train = train[train['DiffTeam']<7]

In [444]:
extra_prob = train[train['DiffTeam']>7]['label'].value_counts(normalize=True)

# Feature Engineering

In [395]:
# train['Inter_feat'] = train_orig['Team1_Venue'] + '_' + train_orig['Team1_Innings']
# test['Inter_feat'] = test_orig['Team1_Venue'] + '_' + test_orig['Team1_Innings']

# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# train['Inter_feat'] = le.fit_transform(train['Inter_feat'])
# test['Inter_feat'] = le.transform(test['Inter_feat'])

# train.head()

,Team1,Team2,Stadium,HostCountry,Team1_Venue,Team2_Venue,Team1_Innings,Team2_Innings,MonthOfMatch,QuarterOfMatch,SemOfMatch,DiffTeam,label,Inter_feat
0,12.679426,9.449761,37,4,1,-1,0,1,12,4,2,3.229665,0,2
1,15.111643,10.446571,84,7,0,0,1,0,9,3,2,4.665072,1,3
2,8.891547,3.468900,47,9,1,-1,1,0,2,1,1,5.422648,1,1
3,0.956938,2.791069,102,6,1,-1,1,0,8,3,2,-1.834131,0,1
4,1.315789,0.598086,46,5,1,-1,1,0,8,3,2,0.717703,1,1


In [396]:
# # Checking whether a team is batting type of team and chasing type of team by checking win percent when they batted first and second
# train_orig_man = train_orig.copy()
# train_orig_man['label'] = train['label']

In [397]:
# def_map = dict(train_orig_man[train_orig['Team1_Innings'] == 'Second'].groupby('Team1')['label'].mean()*10)
# att_map = dict(train_orig_man[train_orig['Team1_Innings'] == 'First'].groupby('Team1')['label'].mean()*10)

# train['Att_rating_Team1'] = train_orig['Team1'].replace(att_map)
# test['Att_rating_Team1'] = test_orig['Team1'].replace(att_map)

# train['Def_rating_Team1'] = train_orig['Team1'].replace(def_map)
# test['Def_rating_Team1'] = test_orig['Team1'].replace(def_map)

# def_map = dict(train_orig_man[train_orig['Team2_Innings'] == 'Second'].groupby('Team1')['label'].mean()*10)
# att_map = dict(train_orig_man[train_orig['Team2_Innings'] == 'First'].groupby('Team1')['label'].mean()*10)

# train['Att_rating_Team2'] = train_orig['Team2'].replace(att_map)
# test['Att_rating_Team2'] = test_orig['Team1'].replace(att_map)

# train['Def_rating_Team2'] = train_orig['Team2'].replace(def_map)
# test['Def_rating_Team2'] = test_orig['Team2'].replace(def_map)

# train['Diff_Att_rating'] = train['Att_rating_Team1'] - train['Att_rating_Team2']
# test['Diff_Att_rating'] = test['Att_rating_Team1'] - test['Att_rating_Team2']

# train['Diff_Def_rating'] = train['Def_rating_Team1'] - train['Def_rating_Team2']
# test['Diff_Def_rating'] = test['Def_rating_Team1'] - test['Def_rating_Team2']

# cols_to_drop = ['Att_rating_Team1','Att_rating_Team2','Def_rating_Team1','Def_rating_Team2']
# train.drop(cols_to_drop,1,inplace=True)
# test.drop(cols_to_drop,1,inplace=True)

In [399]:
# Team1_TargetEnc= train_orig['Team1'].replace(dict(train_orig_man.groupby('Team2')['label'].mean()))
# Team2_TargetEnc = train_orig['Team2'].replace(dict(train_orig_man.groupby('Team2')['label'].mean()))

# test_Team1_TargetEnc = test_orig['Team1'].replace(dict(train_orig_man.groupby('Team2')['label'].mean()))
# test_Team2_TargetEnc = test_orig['Team1'].replace(dict(train_orig_man.groupby('Team2')['label'].mean()))

# train['DiffTargetEnc'] = Team1_TargetEnc - Team2_TargetEnc
# test['DiffTargetEnc'] = test_Team1_TargetEnc - test_Team2_TargetEnc

In [447]:
train.to_csv('Train_pre.csv',index=False)
test.to_csv('Test_pre.csv',index=False)

In [71]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpb4caqfz4
  JVM stdout: /tmp/tmpb4caqfz4/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpb4caqfz4/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.5
H2O_cluster_version_age:,2 days
H2O_cluster_name:,H2O_from_python_unknownUser_8kvip0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [448]:
from h2o.automl import H2OAutoML
df = h2o.import_file('/content/Train_pre.csv')

testing_df = h2o.import_file('/content/Test_pre.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [449]:
#Since this is classification problem, converting target as factor
df['label'] = df['label'].asfactor()

train,test,valid = df.split_frame(ratios=[.85,0.10],seed=11)

y = "label"
x = df.columns
x.remove(y)

In [450]:
df['label'].table()

label,Count
0,1062
1,1299


In [451]:
aml = H2OAutoML(    nfolds=10,
    balance_classes=False,
    class_sampling_factors=None,
    max_after_balance_size=5.0,
    max_runtime_secs=None,
    max_runtime_secs_per_model=None,
    max_models=None,
    stopping_metric='AUTO',
    stopping_tolerance=None,
    stopping_rounds=None,
    seed=4,
    project_name=None,
    exclude_algos=['DeepLearning'],
    include_algos=None,
    exploitation_ratio=0,
    modeling_plan=None,
    monotone_constraints=None,
    algo_parameters=None,
    keep_cross_validation_predictions=True,
    keep_cross_validation_models=False,
    keep_cross_validation_fold_assignment=True,
    sort_metric='logloss',
    export_checkpoints_dir=None,
    verbosity='warn',
)

aml.train(x = x, y = y, training_frame = train, validation_frame=valid)

AutoML progress: |
13:49:37.867: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%


In [452]:
lb = aml.leaderboard # Sub1 = 0.614781, Sub2 = 0.614745 == 0.61567 Balance class is false, Sub3 = 0.612015 == 0.666
lb

model_id,logloss,auc,aucpr,mean_per_class_error,rmse,mse
GBM_5_AutoML_20200621_134937,0.634607,0.661821,0.684088,0.421277,0.471917,0.222705
StackedEnsemble_AllModels_AutoML_20200621_134937,0.635067,0.664969,0.685648,0.413549,0.471948,0.222735
GBM_grid__1_AutoML_20200621_134937_model_6,0.636061,0.66166,0.684838,0.419743,0.472386,0.223149
StackedEnsemble_BestOfFamily_AutoML_20200621_134937,0.636567,0.661514,0.683037,0.417969,0.472687,0.223433
XGBoost_grid__1_AutoML_20200621_134937_model_13,0.63718,0.663038,0.681849,0.419397,0.472887,0.223622
GBM_grid__1_AutoML_20200621_134937_model_5,0.637354,0.656216,0.680326,0.417969,0.473074,0.223799
GBM_grid__1_AutoML_20200621_134937_model_7,0.638559,0.661258,0.680224,0.419503,0.473448,0.224153
XGBoost_grid__1_AutoML_20200621_134937_model_5,0.638973,0.659015,0.676607,0.423261,0.473622,0.224318
GBM_grid__1_AutoML_20200621_134937_model_2,0.639598,0.656372,0.681327,0.415294,0.473929,0.224609
GBM_grid__1_AutoML_20200621_134937_model_4,0.641027,0.664002,0.677836,0.411009,0.474325,0.224984


In [453]:
pred_df = aml.leader.predict(testing_df)
aml.leader.confusion_matrix

gbm prediction progress: |████████████████████████████████████████████████| 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_5_AutoML_20200621_134937


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,32.0,32.0,6736.0,5.0,9.0,6.625,11.0,14.0,12.15625




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.2069903370958405
RMSE: 0.45496190730196356
LogLoss: 0.6008114436858822
Mean Per-Class Error: 0.3126728325531525
AUC: 0.7392814980275649
AUCPR: 0.7583172918907316
Gini: 0.47856299605512986

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.43667663540773693: 


,,0,1,Error,Rate
0,0,275.0,624.0,0.6941,(624.0/899.0)
1,1,60.0,1049.0,0.0541,(60.0/1109.0)
2,Total,335.0,1673.0,0.3406,(684.0/2008.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2020-06-21 13:50:39,3.558 sec,0.0,0.497258,0.687669,0.500000,0.552291,1.000000,0.447709,0.497030,0.687214,0.500000,0.554455,1.000000,0.445545
1,,2020-06-21 13:50:39,3.608 sec,5.0,0.478707,0.650808,0.704682,0.716985,1.435100,0.379482,0.470336,0.634862,0.701587,0.697745,1.803571,0.257426
2,,2020-06-21 13:50:39,3.647 sec,10.0,0.469541,0.632244,0.718411,0.737268,1.724419,0.347112,0.458603,0.610970,0.690278,0.719581,1.803571,0.257426
3,,2020-06-21 13:50:39,3.691 sec,15.0,0.463896,0.620269,0.725982,0.743931,1.724419,0.352590,0.453147,0.598355,0.693452,0.720659,1.803571,0.287129
4,,2020-06-21 13:50:39,3.740 sec,20.0,0.460188,0.612121,0.730305,0.748117,1.724419,0.350100,0.447953,0.586099,0.717063,0.737009,1.803571,0.277228
5,,2020-06-21 13:50:40,3.786 sec,25.0,0.457563,0.606436,0.732765,0.751228,1.724419,0.339641,0.446083,0.580980,0.727778,0.752255,1.803571,0.277228
6,,2020-06-21 13:50:40,3.850 sec,30.0,0.455662,0.602295,0.737919,0.756822,1.724419,0.346614,0.444893,0.577507,0.732937,0.750329,1.803571,0.287129
7,,2020-06-21 13:50:40,3.884 sec,32.0,0.454962,0.600811,0.739281,0.758317,1.724419,0.340637,0.445067,0.577452,0.729762,0.751710,1.803571,0.277228



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,DiffTeam,227.361984,1.000000,0.537581
1,Team1,49.100094,0.215956,0.116094
2,Stadium,38.252945,0.168247,0.090446
3,Team2,32.363796,0.142345,0.076522
4,MonthOfMatch,31.570358,0.138855,0.074646
5,HostCountry,22.626556,0.099518,0.053499
6,Team2_Venue,6.715584,0.029537,0.015879
7,Team2_Innings,5.738923,0.025241,0.013569
8,Team1_Venue,4.759540,0.020934,0.011254
9,Team1_Innings,4.445841,0.019554,0.010512


<bound method H2OBinomialModel.confusion_matrix of >

In [454]:
predictions_prob = pred_df.as_data_frame()
predictions_prob.head()

,predict,p0,p1
0,0,0.712514,0.287486
1,0,0.598771,0.401229
2,0,0.588926,0.411074
3,1,0.410290,0.589710
4,1,0.264501,0.735499


# Submission

In [455]:
test_orig.head()

,Team1,Team2,Stadium,HostCountry,Team1_Venue,Team2_Venue,Team1_Innings,Team2_Innings,MonthOfMatch
0,2,4,34,1,Home,Away,First,Second,Oct
1,14,1,19,15,Home,Away,First,Second,Mar
2,9,10,130,14,Neutral,Neutral,Second,First,Dec
3,9,10,8,9,Home,Away,First,Second,Dec
4,5,15,130,14,Neutral,Neutral,First,Second,Oct


In [456]:
submission = pd.DataFrame(np.zeros(ss.shape),columns = ss.columns)
submission.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [457]:
for i in range(len(test_orig)):
  team1 = predictions_prob.iloc[i]['p1']
  team2 = predictions_prob.iloc[i]['p0']
  submission[test_orig.iloc[i]['Team1']][i] = team1
  submission[test_orig.iloc[i]['Team2']][i] = team2

In [ ]:
for i in range(len(test_orig)):
  

In [458]:
submission.to_excel('Sub6.xlsx',index=False)